In [2]:
from pathlib import Path
import json
import pandas as pd

In [13]:
from pathlib import Path
import json

# Define base datasets path
datasets_path = Path(".")

# Find all config.json files inside any 'collection/*/' subdirectory
config_paths = list(datasets_path.rglob("collection/*/config.json"))

# Collect valid collection IDs based on folder names
valid_collection_ids = {path.parent.name for path in config_paths}

# Load each config.json and annotate with its collection_id
configs = []
for path in config_paths:
    try:
        with path.open("r") as f:
            config_data = json.load(f)
        config_data["collection_id"] = path.parent.name
        configs.append(config_data)
    except Exception as e:
        print(f"Error loading {path}: {e}")

print(f"Loaded {len(configs)} config.json files.")


Loaded 47 config.json files.


In [25]:
all_cql_mosaics = [
    mosaic['cql']
    for config in configs
    if 'mosaic_info' in config and 'mosaics' in config['mosaic_info']
    for mosaic in config['mosaic_info']['mosaics']
    if mosaic.get('cql')  # skips if 'cql' is empty list or missing
]

Only 4 properties are used for searches defined as 'mosaics' in Planetary Computer

In [30]:
from collections import Counter

def extract_properties(cql_expr):
    """Recursively extract all 'property' values from a CQL expression."""
    properties = []
    if isinstance(cql_expr, dict):
        for v in cql_expr.values():
            properties += extract_properties(v)
        if 'property' in cql_expr:
            properties.append(cql_expr['property'])
    elif isinstance(cql_expr, list):
        for item in cql_expr:
            properties += extract_properties(item)
    return properties

# Flatten list of all cql expressions (each is a list of expressions)
property_counter = Counter()
for cql_list in all_cql_mosaics:
    for expr in cql_list:
        props = extract_properties(expr)
        property_counter.update(props)

# Show results
for prop, count in property_counter.most_common():
    print(f"{prop}: {count}")


datetime: 101
eo:cloud_cover: 30
noaa_climate_normals:frequency: 11
noaa_climate_normals:period: 11


In [31]:
from collections import Counter

# Track all top-level keys used inside any CQL expression
cql_keys = Counter()

for config in configs:
    mosaics = config.get('mosaic_info', {}).get('mosaics', [])
    for mosaic in mosaics:
        for expr in mosaic.get('cql', []):
            if isinstance(expr, dict):
                cql_keys.update(expr.keys())

# Display results
print("CQL keys used across all mosaics:")
for key, count in cql_keys.most_common():
    print(f"  {key}: {count}")


CQL keys used across all mosaics:
  op: 153
  args: 153


Every collection has a mosaic_info and render_config in config.json

In [5]:
all_key_sets = [set(config.keys()) for config in configs]
all_keys = set().union(*all_key_sets)
print(all_keys)

{'collection_id', 'mosaic_info', 'render_config'}


In [6]:
all_mosaic_info_key_sets = [set(config.get('mosaic_info',{}).keys()) for config in configs]
all_mosaic_info_keys = set().union(*all_mosaic_info_key_sets)
print(all_mosaic_info_keys)

{'animation_hint', 'default_custom_query', 'mosaics', 'default_location', 'render_options'}


In [18]:
all_mosaic_key_sets = [
    set(mosaic.keys())
    for config in configs
    for mosaic in config.get('mosaic_info',{}).get('mosaics',[])
    ]

mosaic_key_union = set().union(*all_mosaic_key_sets)

print(mosaic_key_union)

{'name', 'description', 'cql'}


In [76]:
all_mosaics = [
    {**mosaic, "collection_id": config['collection_id']}
    for config in configs
    for mosaic in config.get('mosaic_info',{}).get('mosaics',[])
]

all_mosaics_df = pd.DataFrame(all_mosaics)

Commonly, a mosaic includes a cql search based on the properties of datetime, eo:cloud_cover, id, sar:polarizations, or usda_cdl:type

In [ ]:
from collections import Counter

set(all_mosaics_df['cql'].explode().dropna().apply(lambda x: x.get('args')[0]).apply(str))

{"{'property': 'datetime'}",
 "{'property': 'eo:cloud_cover'}",
 "{'property': 'id'}",
 "{'property': 'sar:polarizations'}",
 "{'property': 'usda_cdl:type'}"}

In [84]:
list(all_mosaics_df.apply(lambda x: x.to_dict() if (('sentinel' not in x['collection_id']) and ('datetime' in str(x['cql']))) else None, axis=1).dropna())

[{'name': '2020',
  'description': '2020 Biodiversity Intactness',
  'cql': [{'op': 'anyinteracts',
    'args': [{'property': 'datetime'},
     {'interval': ['2020-01-01T00:00:00Z', '2020-12-31T23:59:59Z']}]}],
  'collection_id': 'io-biodiversity'},
 {'name': '2019',
  'description': '2019 Biodiversity Intactness',
  'cql': [{'op': 'anyinteracts',
    'args': [{'property': 'datetime'},
     {'interval': ['2019-01-01T00:00:00Z', '2019-12-31T23:59:59Z']}]}],
  'collection_id': 'io-biodiversity'},
 {'name': '2018',
  'description': '2018 Biodiversity Intactness',
  'cql': [{'op': 'anyinteracts',
    'args': [{'property': 'datetime'},
     {'interval': ['2018-01-01T00:00:00Z', '2018-12-31T23:59:59Z']}]}],
  'collection_id': 'io-biodiversity'},
 {'name': '2017',
  'description': '2017 Biodiversity Intactness',
  'cql': [{'op': 'anyinteracts',
    'args': [{'property': 'datetime'},
     {'interval': ['2017-01-01T00:00:00Z', '2017-12-31T23:59:59Z']}]}],
  'collection_id': 'io-biodiversity'},


In [ ]:
list(all_mosaics_df.apply(lambda x: x.to_dict() if (('sentinel' not in x['collection_id']) and ('datetime' in str(x['cql']))) else None, axis=1).dropna())

Queryables Can Be Found Using an endpoint

In [32]:
import requests
import json

collections_url = "https://planetarycomputer.microsoft.com/api/stac/v1/collections"
response = requests.get(collections_url)
collections_data = response.json()
all_collection_ids = [collection.get('id') for collection in collections_data["collections"]]
config_collection_ids = [config.get('collection_id') for config in configs]
queryables_list = []

In [33]:
#
for collection_id in valid_collection_ids:
    queryables_url = f"{collections_url}/{collection_id}/queryables"

    try:
        resp = requests.get(queryables_url)
        resp.raise_for_status()
        queryables = resp.json()
        queryables["collection_id"] = collection_id
        queryables_list.append(queryables)
        print(f"Appended queryables for {collection_id}")

    except requests.exceptions.HTTPError as e:
        print(f"Skipping {collection_id}: {e}")

Skipping usgs-gap: 404 Client Error: Not Found for url: https://planetarycomputer.microsoft.com/api/stac/v1/collections/usgs-gap/queryables
Appended queryables for io-biodiversity
Appended queryables for mtbs
Appended queryables for nasadem
Appended queryables for noaa-c-cap
Skipping aster: 404 Client Error: Not Found for url: https://planetarycomputer.microsoft.com/api/stac/v1/collections/aster/queryables
Appended queryables for mobi
Appended queryables for usda-cdl
Appended queryables for ms-buildings
Appended queryables for hgb
Appended queryables for alos-palsar-mosaic
Appended queryables for esa-worldcover
Appended queryables for alos-fnf-mosaic
Appended queryables for alos-dem
Appended queryables for hrea
Appended queryables for sentinel-1-rtc
Appended queryables for drcog-lulc
Appended queryables for nrcan-landcover
Appended queryables for naip
Appended queryables for sentinel-1-grd
Appended queryables for esa-cci-lc
Appended queryables for chloris-biomass
Skipping sentinel-2: 4

In [36]:
queryables_df = pd.DataFrame(queryables_list)

In [53]:
queryables_df['property_key'] = queryables_df['properties'].apply(lambda x: list(x.keys()))

In [71]:
queryables_df

,$id,type,title,$schema,properties,collection_id,property_keys,property_key
0,https://planetarycomputer.microsoft.com/api/st...,object,STAC Queryables.,http://json-schema.org/draft-07/schema#,{'id': {'$ref': 'https://schemas.stacspec.org/...,io-biodiversity,"[id, datetime, geometry, end_datetime, start_d...","[id, datetime, geometry, end_datetime, start_d..."
1,https://planetarycomputer.microsoft.com/api/st...,object,STAC Queryables.,http://json-schema.org/draft-07/schema#,{'id': {'$ref': 'https://schemas.stacspec.org/...,mtbs,"[id, datetime, geometry, end_datetime, start_d...","[id, datetime, geometry, end_datetime, start_d..."
2,https://planetarycomputer.microsoft.com/api/st...,object,STAC Queryables.,http://json-schema.org/draft-07/schema#,{'id': {'$ref': 'https://schemas.stacspec.org/...,nasadem,"[id, datetime, geometry, proj:bbox, proj:epsg,...","[id, datetime, geometry, proj:bbox, proj:epsg,..."
3,https://planetarycomputer.microsoft.com/api/st...,object,STAC Queryables.,http://json-schema.org/draft-07/schema#,{'id': {'$ref': 'https://schemas.stacspec.org/...,noaa-c-cap,"[id, datetime, geometry, end_datetime, start_d...","[id, datetime, geometry, end_datetime, start_d..."
4,https://planetarycomputer.microsoft.com/api/st...,object,STAC Queryables.,http://json-schema.org/draft-07/schema#,{'id': {'$ref': 'https://schemas.stacspec.org/...,mobi,"[id, datetime, geometry, end_datetime, start_d...","[id, datetime, geometry, end_datetime, start_d..."
5,https://planetarycomputer.microsoft.com/api/st...,object,STAC Queryables.,http://json-schema.org/draft-07/schema#,{'id': {'$ref': 'https://schemas.stacspec.org/...,usda-cdl,"[id, datetime, geometry, proj:epsg, proj:shape...","[id, datetime, geometry, proj:epsg, proj:shape..."
6,https://planetarycomputer.microsoft.com/api/st...,object,STAC Queryables.,http://json-schema.org/draft-07/schema#,{'id': {'$ref': 'https://schemas.stacspec.org/...,ms-buildings,"[id, datetime, geometry, end_datetime, start_d...","[id, datetime, geometry, end_datetime, start_d..."
7,https://planetarycomputer.microsoft.com/api/st...,object,STAC Queryables.,http://json-schema.org/draft-07/schema#,{'id': {'$ref': 'https://schemas.stacspec.org/...,hgb,"[id, datetime, geometry, end_datetime, start_d...","[id, datetime, geometry, end_datetime, start_d..."
8,https://planetarycomputer.microsoft.com/api/st...,object,STAC Queryables.,http://json-schema.org/draft-07/schema#,"{'cf': {'type': 'string'}, 'id': {'$ref': 'htt...",alos-palsar-mosaic,"[cf, id, gsd, title, datetime, geometry, platf...","[cf, id, gsd, title, datetime, geometry, platf..."
9,https://planetarycomputer.microsoft.com/api/st...,object,STAC Queryables.,http://json-schema.org/draft-07/schema#,{'id': {'$ref': 'https://schemas.stacspec.org/...,esa-worldcover,"[id, created, mission, datetime, geometry, pro...","[id, created, mission, datetime, geometry, pro..."


In [87]:
all_keys = set().union(*queryables_df['properties'].apply(lambda prop: [key for nested in prop.values() for key in nested.keys()]))


In [97]:
naip_queryables=list(queryables_df[queryables_df['collection_id']=='naip']['properties'])[0]

naip_queryables.keys()

dict_keys(['id', 'gsd', 'datetime', 'geometry', 'naip:year', 'proj:bbox', 'proj:epsg', 'naip:state', 'proj:shape', 'end_datetime', 'proj:transform', 'start_datetime'])

In [105]:
rows = []
for _, row in queryables_df.iterrows():
    collection_id = row['collection_id']
    for prop_name, definition in row['properties'].items():
        rows.append({
            'collection_id': collection_id,
            'property_name': prop_name,
            'definition': definition
        })

properties_df = pd.DataFrame(rows)

In [ ]:
definitions_df = properties_df.map(str).groupby('property_name').agg(lambda x: list(set(x))).reset_index()[['property_name','definition']]\
    .explode('definition').map(lambda x: eval(x) if '{' in x else x)\
    .apply(lambda x: {'property_name':x.get('property_name'), 'definition':list(x.get('definition').items())}, axis=1)\
    .apply(pd.Series).explode('definition')

definitions_df['']

In [ ]:
missing_queryable_definitions={'example', 'examples', 'items', 'pattern', 'integer', 'type': 'sar:looks_range'}

In [159]:
definitions_df['key']=definitions_df['definition'].apply(lambda x: str(x[0]))
definitions_df['value']=definitions_df['definition'].apply(lambda x: str(x[1]))
pd.DataFrame(definitions_df.groupby(['key','value'])['property_name'].agg(list).head(90).tail(30))

property_name
key   value                                                                       
title Orbit Source                                               [s1:orbit_source]
      Orbit State                                                [sat:orbit_state]
      Pixel spacing azimuth (m)                        [sar:pixel_spacing_azimuth]
      Pixel spacing range (m)                            [sar:pixel_spacing_range]
      Platform                                                [platform, platform]
      Platform Designation                 [sat:platform_international_designator]
      Polarization number                         [palsar:number_of_polarizations]
      Polarizations                                            [sar:polarizations]
      Product Timeliness                                   [s1:product_timeliness]
      Product Version                                         [esa_cci_lc:version]
      Product type                                              [sar:product_type]
      Proposed Data Citation                                        [sci:citation]
      Relative Orbit                                          [sat:relative_orbit]
      Resolution                                                   [s1:resolution]
      Resolution azimuth (m)                              [sar:resolution_azimuth]
      Resolution range (m)                                  [sar:resolution_range]
      Shape                                                           [proj:shape]
      Slice Number                                               [s1:slice_number]
      Start datetime                                              [start_datetime]
      State                                                           [naip:state]
      Tile                                                       [esa_cci_lc:tile]
      Tile ID                                        [esa_worldcover:product_tile]
      Total Slices                                               [s1:total_slices]
      Transform                                                   [proj:transform]
      Year                                                             [naip:year]
type  ['integer', 'null']                                              [proj:epsg]
      array                      [instruments, proj:bbox, proj:shape, proj:tran...
      integer                                 [sar:looks_azimuth, sar:looks_range]
      number                     [gsd, sar:center_frequency, sar:looks_equivale...
      string                     [cf, constellation, created, datetime, descrip...

In [57]:
queryable_properties_df = queryables_df[['collection_id','property_key']].explode('property_key')\
    .groupby('property_key').agg(list).reset_index()

queryable_properties_df['collection_count']=queryable_properties_df['collection_id'].apply(lambda x: len(set(x)))

queryable_properties_df.sort_values(by='collection_count', ascending=False)

,property_key,collection_id,collection_count
11,id,"[io-biodiversity, mtbs, nasadem, noaa-c-cap, m...",21
51,start_datetime,"[io-biodiversity, mtbs, nasadem, noaa-c-cap, m...",21
3,datetime,"[io-biodiversity, mtbs, nasadem, noaa-c-cap, m...",21
5,end_datetime,"[io-biodiversity, mtbs, nasadem, noaa-c-cap, m...",21
9,geometry,"[io-biodiversity, mtbs, nasadem, noaa-c-cap, m...",21
20,proj:epsg,"[nasadem, usda-cdl, alos-palsar-mosaic, esa-wo...",12
21,proj:shape,"[nasadem, usda-cdl, alos-palsar-mosaic, alos-f...",10
22,proj:transform,"[nasadem, usda-cdl, alos-palsar-mosaic, alos-f...",10
19,proj:bbox,"[nasadem, alos-palsar-mosaic, alos-fnf-mosaic,...",6
10,gsd,"[alos-palsar-mosaic, alos-fnf-mosaic, alos-dem...",4


All type, title, and $schema are the same

In [43]:
{k: set(queryables_df[k]) for k in ['type', 'title', '$schema']}

{'type': {'object'},
 'title': {'STAC Queryables.'},
 '$schema': {'http://json-schema.org/draft-07/schema#'}}